In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv("/Users/anuraagjain/Downloads/gender-classifier-DFE-791531.csv", encoding='latin1')

In [ ]:
# Utility function taken from: https://github.com/rasto2211/Twitter-User-Gender-Classification/blob/master/notebooks/exploration.ipynb
# Normalizes text for analysis by removing URLs, special characters, and double spaces

def normalize_text(text):
    # Remove non-ASCII chars.
    text = re.sub('[^\x00-\x7F]+',' ', text)
    
    # Remove URLs
    text = re.sub('https?:\/\/.*[\r\n]*', ' ', text)
    
    # Remove special chars.
    text = re.sub('[?!+%{}:;.,"\'()\[\]_]', '',text)
    
    # Remove double spaces.
    text = re.sub('\s+',' ',text)
    
    return text

df['edited_text'] = [normalize_text(text) for text in df['text']]

print(df['edited_text'])

In [ ]:
# Choose data only where gender is either male or female and gender classification confidence is about 1
chosen_rows = df[df["gender"].isin(["male", "female"]) & (df["gender:confidence"] > 0.99)].index.tolist()

# Shuffle data to ensure randomness
random.shuffle(chosen_rows)

n = len(chosen_rows)
train_data_size = .6
test_data_size = .2
validation_data_size = .2

# Partition chosen_rows
train_data_nrows = round(train_data_size * n)
train_data = chosen_rows[:train_data_nrows]
validation_data_upper_limit = (train_data_nrows + round(validation_data_size * n))
validation_data = chosen_rows[train_data_nrows : validation_data_upper_limit]
test_data = chosen_rows[validation_data_upper_limit:]

In [ ]:
# Making classifier
train_counts = CountVectorizer().fit_tr